In [1]:
import xml.etree.ElementTree as ET
import numpy as np

def all_xml_to_np(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    if root.tag != "annotation":
        raise Exception("pascal voc xml root element should be annotation, rather than {}".format(root.tag))
    gt_boxes = []
    for elem in root.findall("object/bndbox"):
        startrow = int(elem.find("startrow").text)
        endrow = int(elem.find("endrow").text)
        startcol = int(elem.find("startcol").text)
        endcol = int(elem.find("endcol").text)
        xmin = float(elem.find("xmin").text)
        ymin = float(elem.find("ymin").text)
        xmax = float(elem.find("xmax").text)
        ymax = float(elem.find("ymax").text)

        x1 = float(elem.find("x1").text)
        y1 = float(elem.find("y1").text)
        x2 = float(elem.find("x2").text)
        y2 = float(elem.find("y2").text)
        x3 = float(elem.find("x3").text)
        y3 = float(elem.find("y3").text)
        x4 = float(elem.find("x4").text)
        y4 = float(elem.find("y4").text)

        gt_boxes.append(
            [x1, y1, x2, y2, x3, y3, x4, y4, xmin, ymin, xmax, ymax, startrow, endrow, startcol, endcol]
        )
    np_gt_boxes = np.array(gt_boxes)
    np.set_printoptions(precision=2, suppress=True)
    return np_gt_boxes

In [1]:
import os.path as osp
from sys import argv
import cv2
import mmcv
import wandb
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector

# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
import subprocess
import random

wandb.login()

cfg = Config.fromfile(f"/home/aiarhipov/centernet/exps/27_seg/config.py")


set_random_seed(0, deterministic=False)
cfg.gpu_ids = [1, 2]
val = True
# Build dataset

# datasets = build_dataset(cfg.data.train)
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=True, validate=val)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: archiealexarkhipov (centernet). Use `wandb login --relogin` to force relogin


loading annotations into memory...
Done (t=12.84s)
creating index...
index created!
loading annotations into memory...
Done (t=3.71s)
creating index...
index created!


KeyError: 'LOCAL_RANK'

In [ ]:
import json
import os
import xml.etree.ElementTree as ET

coco = dict()
coco["images"] = []
coco["type"] = "instances"
coco["annotations"] = []
coco["categories"] = []

category_set = dict()
image_set = set()

category_item_id = 0
image_id = 20140000000
annotation_id = 0


def addCatItem(name):
    global category_item_id
    category_item = dict()
    category_item["supercategory"] = "none"
    category_item_id += 1
    category_item["id"] = category_item_id
    category_item["name"] = name
    coco["categories"].append(category_item)
    category_set[name] = category_item_id
    return category_item_id


def addImgItem(file_name, size):
    global image_id
    if file_name is None:
        raise Exception("Could not find filename tag in xml file.")
    if size["width"] is None:
        raise Exception("Could not find width tag in xml file.")
    if size["height"] is None:
        raise Exception("Could not find height tag in xml file.")
    image_id += 1
    image_item = dict()
    image_item["id"] = image_id
    image_item["file_name"] = file_name
    image_item["width"] = size["width"]
    image_item["height"] = size["height"]
    coco["images"].append(image_item)
    image_set.add(file_name)
    return image_id


def addAnnoItem(object_name, image_id, category_id, bbox, seg):
    global annotation_id
    annotation_item = dict()
    annotation_item["segmentation"] = []

    annotation_item["segmentation"].append(seg)

    annotation_item["area"] = bbox[2] * bbox[3]
    annotation_item["iscrowd"] = 0
    annotation_item["ignore"] = 0
    annotation_item["image_id"] = image_id
    annotation_item["bbox"] = bbox
    annotation_item["category_id"] = category_id
    annotation_id += 1
    annotation_item["id"] = annotation_id
    coco["annotations"].append(annotation_item)


def parseXmlFiles(XML_path, four_boxes=False, div=1, ddiv=1):
    for xml_path in XML_path:
        for idx, f in enumerate(os.listdir(xml_path)):
            if idx % div == 0:
                if not f.endswith(".xml"):
                    continue

                bndbox = dict()
                size = dict()
                current_image_id = None
                current_category_id = None
                file_name = None
                size["width"] = None
                size["height"] = None
                size["depth"] = None

                xml_file = os.path.join(xml_path, f)
                print(xml_file)

                tree = ET.parse(xml_file)
                root = tree.getroot()
                if root.tag != "annotation":
                    raise Exception(
                        "pascal voc xml root element should be annotation, rather than {}".format(root.tag)
                    )

                # elem is <folder>, <filename>, <size>, <object>
                for elem in root:
                    current_parent = elem.tag
                    current_sub = None
                    object_name = None

                    if elem.tag == "folder":
                        continue

                    if elem.tag == "filename":
                        file_name = elem.text
                        if file_name in category_set:
                            raise Exception("file_name duplicated")

                    # add img item only after parse <size> tag
                    elif current_image_id is None and file_name is not None and size["width"] is not None:
                        if file_name not in image_set:
                            current_image_id = addImgItem(file_name, size)
                        else:
                            raise Exception("duplicated image: {}".format(file_name))
                    # subelem is <width>, <height>, <depth>, <name>, <bndbox>
                    for idx2, subelem in enumerate(elem):
                        bndbox["xmin"] = None
                        bndbox["xmax"] = None
                        bndbox["ymin"] = None
                        bndbox["ymax"] = None

                        current_sub = subelem.tag
                        if current_parent == "object" and subelem.tag == "name":
                            object_name = subelem.text
                            if object_name not in category_set:
                                current_category_id = addCatItem(object_name)
                            else:
                                current_category_id = category_set[object_name]

                        elif current_parent == "size":
                            if size[subelem.tag] is not None:
                                raise Exception("xml structure broken at size tag.")
                            size[subelem.tag] = int(subelem.text)

                        # option is <xmin>, <ymin>, <xmax>, <ymax>, when subelem is <bndbox>
                        filter = {
                            "x1": 0,
                            "x2": 0,
                            "x3": 0,
                            "x4": 0,
                            "y1": 0,
                            "y2": 0,
                            "y3": 0,
                            "y4": 0,
                        }  # ,'x5':0,'y5':0,'x6':0,'y6':0}
                        seg = []
                        for option in subelem:
                            if current_sub == "bndbox":
                                if option.tag in filter.keys():
                                    filter[option.tag] = float(option.text)
                                # elif bndbox[option.tag] is not None:
                                #    raise Exception('xml structure corrupted at bndbox tag.')
                                bndbox[option.tag] = float(option.text)
                        seg = [
                            filter["x1"],
                            filter["y1"],
                            filter["x2"],
                            filter["y2"],
                            filter["x3"],
                            filter["y3"],
                            filter["x4"],
                            filter["y4"],
                        ]  # ,filter['x5'],filter['y5'],filter['x6'],filter['y6']]

                        # only after parse the <object> tag
                        if bndbox["xmin"] is not None:
                            if object_name is None:
                                raise Exception("xml structure broken at bndbox tag")
                            if current_image_id is None:
                                raise Exception("xml structure broken at bndbox tag")
                            if current_category_id is None:
                                raise Exception("xml structure broken at bndbox tag")
                            if four_boxes:
                                four+=4
                                if idx2 % ddiv == 0:
                                    bbox = []
                                    center_x = (seg[0] + seg[2] + seg[4] + seg[6]) / 4
                                    center_y = (seg[1] + seg[3] + seg[5] + seg[7]) / 4
                                    bbox.append(center_x)
                                    bbox.append(center_y)
                                    bbox.append(-seg[0])
                                    bbox.append(-seg[1])
                                    addAnnoItem(
                                        object_name,
                                        current_image_id,
                                        current_category_id,
                                        bbox,
                                        seg,
                                    )
                                    bbox = []
                                    bbox.append(center_x)
                                    bbox.append(center_y)
                                    bbox.append(seg[2])
                                    bbox.append(-seg[3])
                                    addAnnoItem(
                                        object_name,
                                        current_image_id,
                                        current_category_id,
                                        bbox,
                                        seg,
                                    )
                                    bbox = []
                                    bbox.append(center_x)
                                    bbox.append(center_y)
                                    bbox.append(seg[4])
                                    bbox.append(seg[5])
                                    addAnnoItem(
                                        object_name,
                                        current_image_id,
                                        current_category_id,
                                        bbox,
                                        seg,
                                    )
                                    bbox = []
                                    bbox.append(center_x)
                                    bbox.append(center_y)
                                    bbox.append(-seg[6])
                                    bbox.append(seg[7])
                                    addAnnoItem(
                                        object_name,
                                        current_image_id,
                                        current_category_id,
                                        bbox,
                                        seg,
                                    )
                            else:
                                bbox = []
                                # x
                                bbox.append(bndbox["xmin"])
                                # y
                                bbox.append(bndbox["ymin"])
                                # w
                                bbox.append(bndbox["xmax"] - bndbox["xmin"])
                                # h
                                bbox.append(bndbox["ymax"] - bndbox["ymin"])
                                addAnnoItem(
                                    object_name,
                                    current_image_id,
                                    current_category_id,
                                    bbox,
                                    seg,
                                )



    DIV = 4
    xml_path = ["/home/aiarhipov/datasets/WTW-dataset/train/xml"]
    json_file = f"/home/aiarhipov/datasets/WTW-dataset/train/train_4x{10}.json"
    parseXmlFiles(xml_path, four_boxes=True, div=DIV, ddiv=2)

    # xml_path = ["/home/aiarhipov/datasets/WTW-dataset/test/xml"]
    # json_file = f"/home/aiarhipov/datasets/WTW-dataset/test/test_4x{10}.json"
    # parseXmlFiles(xml_path, four_boxes=True, div=DIV, ddiv=2)
    # json.dump(coco, open(json_file, "w"))